In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [7]:
import yfinance as yf

ticker = "GBPUSD=X"
start_date = "2005-01-01"
end_date = "2025-01-01"

print(f"Fetching data for {ticker}...")
data = yf.download(ticker, start=start_date, end=end_date)

# Select only the 'Close' column
df = data[['Close']]

# The rename line is now removed
# df.rename(columns={'Close': 'Price'}, inplace=True)

filename = 'gbpusd_20y.csv'
df.to_csv(filename)

print(f"Data saved successfully to {filename} with 'Close' column.")

#need to manually edit the csv file to remove the first row of names

Fetching data for GBPUSD=X...


/var/folders/m6/mgjfjp2s2b97v0p3wmv8x69m0000gn/T/ipykernel_52859/1812043836.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Data saved successfully to gbpusd_20y.csv with 'Close' column.


In [15]:
df = pd.read_csv('gbpusd_20y.csv', parse_dates=True, index_col=0)
df = df[['Close']]

In [16]:
df.head()

,Close
Date,
2005-01-03,1.904617
2005-01-04,1.883594
2005-01-05,1.885512
2005-01-06,1.876490
2005-01-07,1.871293


In [17]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# 创建序列数据
def create_dataset(data, look_back=60):
    X, y = [], []
    for i in range(len(data)-look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

look_back = 60
X, y = create_dataset(scaled_data, look_back)

# 划分训练集与测试集（80%训练，20%测试）
split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 转换为LSTM需要的格式 [样本数, 时间步数, 特征数]
X_train = np.reshape(X_train, (X_train.shape[0], look_back, 1))
X_test = np.reshape(X_test, (X_test.shape[0], look_back, 1))

In [18]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

/Users/snowyan/gbptousd/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)